# Downloading dataset from kaggle using kaggle data api

In [ ]:
#connect to kaggle for dataset downlord
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

#downlord dataset
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

#unzip dataset
!unzip imdb-dataset-of-50k-movie-reviews.zip

#reading only top SOME SAMPLE rows
import pandas as pd
df = pd.read_csv('IMDB Dataset.csv').head(50000)


 97% 25.0M/25.7M [00:00<00:00, 82.5MB/s]
100% 25.7M/25.7M [00:00<00:00, 88.3MB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


#Preparing dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


X = df['review'].values
y = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values

vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(X).toarray()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Define constants
input_size = X_train.shape[1]
hidden_size = 128
output_size = 2
learning_rate = 0.001
batch_size = 64
epochs = 10

#Model Architecture

While models like RNNs and LSTMs have been widely used for sequence modeling tasks, TCNs present an alternative that combines efficiency, parallelization, and the ability to capture long-range dependencies, making them suitable for tasks like sentiment analysis where contextual information matters.

In [ ]:
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1)
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)

    def forward(self, x):
        return self.net(x)



class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class SentimentTCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(SentimentTCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)

    def forward(self, x):
        y = self.tcn(x.unsqueeze(2))  # Adjust input dimensions
        y = y[:, :, -1]  # Take the output at the last time step
        return self.linear(y)

#Training and Evaluation

In [ ]:
print(input_size)
# Instantiate the model
model = SentimentTCN(input_size, output_size, num_channels=[hidden_size]*4, kernel_size=2, dropout=0.2)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create DataLoader for training and validation sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Training loop
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f'Epoch {epoch+1}/{epochs}, Loss: {val_loss/len(val_loader)}, Accuracy: {accuracy}')

5000


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Epoch 1/10, Loss: 0.28411131565737874, Accuracy: 0.8833
Epoch 2/10, Loss: 0.2813898280357859, Accuracy: 0.8869
Epoch 3/10, Loss: 0.27964116666157535, Accuracy: 0.8927
Epoch 4/10, Loss: 0.3084000036310238, Accuracy: 0.8875
Epoch 5/10, Loss: 0.3635955649386546, Accuracy: 0.8845
Epoch 6/10, Loss: 0.44176918913604346, Accuracy: 0.8848
Epoch 7/10, Loss: 0.49341675079172587, Accuracy: 0.8823
Epoch 8/10, Loss: 0.49301986961038247, Accuracy: 0.8804
Epoch 9/10, Loss: 0.5062911537991968, Accuracy: 0.8833
Epoch 10/10, Loss: 0.5443131752834198, Accuracy: 0.8794
